
# **Conclusion**

<a name="Dataset"></a>
## Dataset


> Used Cars
<p><img alt="Colaboratory logo" height="55px" src="https://upload.wikimedia.org/wikipedia/commons/thumb/9/99/GAZ-24_Volga.svg/640px-GAZ-24_Volga.svg.png" align="left" hspace="10px" vspace="0px"></p>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [1]:
import pandas as pd
data = pd.read_csv("/content/drive/MyDrive/Python Course/ann_used_cars.csv")
data.head()

FileNotFoundError: ignored

<a class="anchor" id="first-bullet"></a>
<img src="https://i.ibb.co/nzdQxG1/car.png" alt="Drawing" style="width: 600px;"/>

In [ ]:
#! pip install scikeras[tensorflow]

In [ ]:
data = pd.get_dummies(data)
data.head()

,year,price,mileage,model_SE,model_SEL,model_SES,color_Black,color_Blue,color_Gold,color_Gray,color_Green,color_Red,color_Silver,color_White,color_Yellow,transmission_AUTO,transmission_MANUAL
0,2011,21992,7413,0,1,0,0,0,0,0,0,0,0,0,1,1,0
1,2011,20995,10926,0,1,0,0,0,0,1,0,0,0,0,0,1,0
2,2011,19995,7351,0,1,0,0,0,0,0,0,0,1,0,0,1,0
3,2011,17809,11613,0,1,0,0,0,0,1,0,0,0,0,0,1,0
4,2012,17500,8367,1,0,0,0,0,0,0,0,0,0,1,0,1,0


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.drop('price', axis=1), data['price'], test_size=0.2, random_state=1)

In [ ]:
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size = 0.5)

<h1> Normalizing </h1>
<a href="https://ibb.co/BsC7CgQ"><img src="https://i.ibb.co/ThgngH7/zscore.png" alt="zscore" border="0"></a>

In [ ]:
# Z-score Normalizing
from sklearn import preprocessing
X_train = preprocessing.scale(X_train)
X_val = preprocessing.scale(X_val)
X_test = preprocessing.scale(X_test)

<h1> Hyperparameter Tuning </h1>
<a href="https://imgbb.com/"><img src="https://i.ibb.co/W3WZC8d/tune.png" alt="tune" border="0"></a>

In [ ]:
!pip install scikeras


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, InputLayer
#from keras.wrappers.scikit_learn import KerasClassifier
from scikeras.wrappers import KerasClassifier,KerasRegressor
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.optimizers import Adam, SGD
import warnings
warnings.filterwarnings("ignore")

Define the grid search parameters


In [ ]:
batch_size=[10,100]
epochs=[10,100]
hidden_unit1= [1,64]
hidden_unit2 = [1,64]
optimizer = ["SGD","Adam"]

hyperparameters = dict(
                       batch_size = batch_size,
                       epochs=epochs,
                       hidden_unit1=hidden_unit1,
                       hidden_unit2=hidden_unit2,
                       optimizer=optimizer
                       )

In [ ]:
hyperparameters

{'batch_size': [10, 100],
 'epochs': [10, 100],
 'hidden_unit1': [1, 64],
 'hidden_unit2': [1, 64],
 'optimizer': ['SGD', 'Adam']}

Function to create model, required for KerasClassifier

In [ ]:
def create_model(hidden_unit1,hidden_unit2,optimizer='Adam'):
  model = tf.keras.Sequential()
  model.add(InputLayer(input_shape=X_train.shape[1]))
  model.add(Dense(hidden_unit1, activation = 'relu'))
  model.add(Dense(hidden_unit2, activation = 'relu'))
  model.add(Dense(1, activation='relu'))
  #optimizer = optimizer()
  model.compile(loss='mean_squared_error', optimizer = optimizer, metrics=[tf.keras.metrics.MeanSquaredError()])

  return model

Wrap model so it can be used by scikit-learn


In [ ]:
neural_network = KerasRegressor(build_fn=create_model,hidden_unit1=hidden_unit1,hidden_unit2=hidden_unit2, optimizer=optimizer,batch_size = batch_size,epochs=epochs)

Conduct Grid Search

In [ ]:
# Create grid search
grid = GridSearchCV(estimator=neural_network, param_grid=hyperparameters, n_jobs=-1, cv=5)

# Fit grid search
grid_result = grid.fit(X_train, y_train)

Epoch 1/100
12/12 [==============================] - 2s 3ms/step - loss: 176755120.0000 - mean_squared_error: 176755120.0000
Epoch 2/100
12/12 [==============================] - 0s 3ms/step - loss: 176737376.0000 - mean_squared_error: 176737376.0000
Epoch 3/100
12/12 [==============================] - 0s 3ms/step - loss: 176709264.0000 - mean_squared_error: 176709264.0000
Epoch 4/100
12/12 [==============================] - 0s 3ms/step - loss: 176673072.0000 - mean_squared_error: 176673072.0000
Epoch 5/100
12/12 [==============================] - 0s 3ms/step - loss: 176622128.0000 - mean_squared_error: 176622128.0000
Epoch 6/100
12/12 [==============================] - 0s 3ms/step - loss: 176550608.0000 - mean_squared_error: 176550608.0000
Epoch 7/100
12/12 [==============================] - 0s 3ms/step - loss: 176453552.0000 - mean_squared_error: 176453552.0000
Epoch 8/100
12/12 [==============================] - 0s 4ms/step - loss: 176321040.0000 - mean_squared_error: 176321040.0000


In [ ]:
# summarize results
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

<a name="Training"></a>
## Training


<a href="https://ibb.co/qntkSS8"><img src="https://i.ibb.co/n0HzGGy/gd.png" height="250px" alt="gd" border="0"></a>

In [ ]:
# Fitting the model with the best set of identified parameters
model = tf.keras.Sequential()
model.add(InputLayer(input_shape=X_train.shape[1]))
model.add(Dense(grid_result.best_params_['hidden_unit1'], activation = 'relu'))
model.add(Dense(grid_result.best_params_['hidden_unit2'], activation = 'relu'))
model.add(Dense(1, activation='relu', bias_initializer=tf.keras.initializers.Ones()))

model.compile(loss='mean_squared_error', optimizer= grid_result.best_params_['optimizer'], metrics=[tf.keras.metrics.MeanSquaredError()])
history = model.fit(X_train, y_train,  batch_size = grid_result.best_params_['batch_size'], epochs = grid_result.best_params_['epochs'], verbose = 1, validation_data=(X_val,y_val))

In [ ]:
# Plot Learning Curve
import matplotlib.pyplot as plt
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='validate')
plt.xlabel("epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
y_pred = model.predict(X_test)
print('Predicted Values', y_pred[0:5].round().tolist())
print('Actual Values', y_test[0:5].tolist())

In [ ]:
# Accuracy (Root Mean Squared Error)
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, y_pred, squared=False)